### tokenized_data_SlimPajama

In [4]:
import os
import pandas as pd
from hashlib import sha256
from datasets import load_dataset
from transformers import LlamaTokenizer
from multiprocessing import Pool


def tokenize_data(text_id):
    """Tokenize text data and generate ID."""
    text, tokenizer = text_id
    tokenized_text = tokenizer(text, truncation=True, max_length=1000)
    # Convert BatchEncoding to dictionary and extract necessary fields
    tokenized_text_data = {
        'input_ids': tokenized_text['input_ids'],

    }
    return tokenized_text_data

def filter_by_word_count(example):
    """Calculate the number of words in the text."""
    word_count = len(example['text'].split())
    return 50 < word_count < 800

def main(data_row):
    # Load dataset
    dataset = load_dataset("DKYoon/SlimPajama-6B", split='train')
    # Take the first 1 million rows that meet the word count criteria
    # num = 5000000
    num = data_row
    dataset = dataset.select(range(num)).filter(filter_by_word_count)
    print(len(dataset))
    # Load tokenizer
    tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

    # Prepare data for multiprocessing
    texts = [(text, tokenizer) for text in dataset['text']]  # Assuming 'text' field exists

    # Use multiprocessing to tokenize data
    with Pool(os.cpu_count()) as pool:
        results = pool.map(tokenize_data, texts)

    # Convert results to DataFrame and save as Parquet file
    df = pd.DataFrame(results)
    df.to_parquet('tokenized_data_SlimPajama.parquet')

    print("Data tokenized and saved successfully as Parquet.")

if __name__ == "__main__":
    
    main(5000000)


Filter: 100%|██████████| 40/40 [00:00<00:00, 6704.99 examples/s]
/home/yuhao/anaconda/envs/easycontext/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


33
Data tokenized and saved successfully as Parquet.


### Tokenized_data_Orca

In [5]:
import os
import pandas as pd
from hashlib import sha256
from datasets import load_dataset
from transformers import LlamaTokenizer
from multiprocessing import Pool

def filter_criteria(example):
    # 计算单词数量
    question_words = len(example['question'].split())
    response_words = len(example['response'].split())
    # 检查 id 是否以 'cot' 开头
    question_lower = example['question'].lower()
    contains_translate = "translate" in question_lower or "translation" in question_lower
    
    # 判断单词数量是否都小于1000且question中不含"translate"和"translation"
    return 80 < question_words < 600 and 80 < response_words < 600 and not contains_translate


def tokenize_data(example, tokenizer):
    """Tokenize question and response data."""
    tokenized_question = tokenizer(example['question'])
    tokenized_response = tokenizer(example['response'])
    return {
        'question_input_ids': tokenized_question['input_ids'],
        'response_input_ids': tokenized_response['input_ids']
    }

def main():
    # Load dataset
    dataset = load_dataset("Open-Orca/OpenOrca", split='train')
    # Filter dataset based on criteria
    filtered_dataset = dataset.filter(filter_criteria)
    print(filtered_dataset)

    # Load tokenizer
    tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

    # Prepare data for multiprocessing
    pool = Pool(os.cpu_count())
    results = [pool.apply_async(tokenize_data, (example, tokenizer)) for example in filtered_dataset]
    results = [result.get() for result in results]
    pool.close()
    pool.join()

    # Convert results to DataFrame and save as Parquet file
    df = pd.DataFrame(results)
    df.to_parquet('tokenized_data_Orca_all.parquet')

    print("Data tokenized and saved successfully as Parquet.")

if __name__ == "__main__":
    main()


Filter: 100%|██████████| 4233923/4233923 [01:09<00:00, 61195.89 examples/s]


Dataset({
    features: ['id', 'system_prompt', 'question', 'response'],
    num_rows: 534723
})


KeyboardInterrupt: 

Dataset({
    features: ['text', 'meta', '__index_level_0__'],
    num_rows: 5489000
})